In [63]:
import pulp
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import itertools
from copy import deepcopy as dcopy

In [64]:
# Working through the hooks puzzles.
# Different to #2 as many more limitations on potential hook placements. 24 instead of 3200
# need to work out a good method of solving !

### Puzzle details
<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2018/02/hook3_puzzle-01.png" width="300" height="300">

In [105]:
#Setup the constraints
top_labels = [0,35,42,18,18,0, 36, 63,0]
bot_labels = [0,40,32,40,10,12,0,56,0]
left_labels = [0,56,0,32,40,15,16,25,0]
right_labels =[0,49,63,0,18,42,63,54,0]
params = [top_labels,bot_labels,left_labels,right_labels]

In [139]:
class Matrix():
    def __init__(self,top_labels,bot_labels,left_labels,right_labels):
        self.top_labels = top_labels
        self.bot_labels = bot_labels
        self.left_labels = left_labels
        self.right_labels = right_labels
        self.potential_grids = [[-9,np.ones((9,9),dtype=int)*-1,[0,0,9,9]]] # list of all grids not yet ruled out.[Level,grid,coordinates of the grid yet to be filled]
        self.solution = []
        self.splits =  [list(i) for i in itertools.product([0, 1], repeat=9) if sum(i) > 1]
     
    def add_layer(self,grid,coords,lvl,alignment):
        row_start,col_start,row_end, col_end = coords
    
        if alignment == 0:
            grid[row_start:row_end,col_start] =lvl
            grid[row_start,col_start:col_end] =lvl
            row_start +=1
            col_start +=1
    
        if alignment == 1:
            grid[row_start:row_end,col_start] =lvl
            grid[row_end-1,col_start:col_end] =lvl
            row_end -=1
            col_start +=1 

        if alignment == 2:
            grid[row_start:row_end,col_end-1] =lvl
            grid[row_start,col_start:col_end] =lvl
            row_start +=1
            col_end -=1
        
        if alignment == 3:
            grid[row_start:row_end,col_end-1] =lvl
            grid[row_end-1,col_start:col_end] =lvl
            row_end -=1
            col_end -=1 
    
        coords = [row_start,col_start,row_end, col_end]
        
        return grid,coords    
   
    def check_grid(self,grid):
        isValid = 1
        for i in range(9):
            row = grid[i,:]
            col = grid[:,i]
            if -1 not in row:
                    isValid *= self.check_line(row,self.left_labels[i],self.right_labels[i])
            if -1 not in col:
                    isValid *= self.check_line(col,self.top_labels[i],self.bot_labels[i])  
        return isValid
                    
    def check_line(self,line,start,end):
        for split in self.splits:
            test = line * split
            if (test[test!=0][:2].prod() == start) or (start ==0):
                if (test[test!=0][-2:].prod() == end) or (end ==0):
                    return 1
        return 0
    
    def solve(self):
            
        while len(self.potential_grids) > 0:
            
            temp_grid = self.potential_grids.pop(0)
            #create the potential rotations at the given level
            rotations = []
        
            for alignment in range(4):
                lvl,grid,coords = dcopy(temp_grid)
                grid,coords = self.add_layer(grid,coords,-lvl,alignment)
                if lvl != -1 :
                    rotations.append([lvl+1,grid,coords])
                else:
                     rotations = [[lvl+1,grid,coords]]
    
            
            #check valid grids (where the sum can be made from available digits) and save the ones that work
            for i in range(len(rotations)):
                lvl,g,coords = rotations[i]
                if self.check_grid(g):
                    if lvl !=0:    
                        self.potential_grids.append([lvl,g,coords])
                    else:
                        self.solution.append(g)
                        
        print("There are {} valid hooks".format(len(self.solution)))
        
        #solve each grid in the cut down list
        for i in range(len(self.solution)):
            grid = self.solution[i]
            #solved,matrix = self.solve_matrix(grid)
            
            #if np.sum(np.abs(np.sum(solved*matrix,axis=1)-self.row_labels)) + np.sum(np.abs(np.sum(solved*matrix,axis=0)-self.col_labels)) == 0:
            #    self.final = [solved,matrix]
            #    break
                
def sol_print(solved,matrix):
    fig,ax = plt.subplots(1,1,figsize=(5,5))
    ax = sns.heatmap(matrix,annot=solved*matrix,cbar=False,cmap="YlGnBu")
    ax.axis("off")
    
    

In [140]:
start = time.perf_counter()
test = Matrix(top_labels,bot_labels,left_labels,right_labels)
test.solve()
#solved,matrix = test.final
stop =  time.perf_counter()
print('Solution took {:0.4f} seconds\n'.format((stop-start)))
#sol_print(solved,matrix)

There are 24 valid hooks
Solution took 0.8307 seconds



### Puzzle solution
<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2018/03/20180228_hooks_3_ans.png" width="300" height="400">

In [122]:
correct = np.array([[9, 9, 9, 9, 9, 9, 9, 9, 9],
                    [8, 7, 7, 7, 7, 7, 7, 7, 9],
                    [8, 6, 6, 6, 6, 6, 6, 7, 9],
                    [8, 5, 4, 3, 2, 1, 6, 7, 9],
                    [8, 5, 4, 3, 2, 2, 6, 7, 9],
                    [8, 5, 4, 3, 3, 3, 6, 7, 9],
                    [8, 5, 4, 4, 4, 4, 6, 7, 9],
                    [8, 5, 5, 5, 5, 5, 6, 7, 9],                          
                    [8, 8, 8, 8, 8, 8, 8, 8, 9]])

solved=   np.array([[0, 0, 0, 0, 9, 9, 0, 9, 0],
                    [8, 7, 7, 0, 0, 7, 0, 7, 0],
                    [0, 0, 6, 6, 0, 6, 6, 7, 9],
                    [8, 0, 4, 0, 0, 1, 0, 0, 9],
                    [8, 5, 4, 3, 2, 2, 0, 0, 9],
                    [0, 5, 0, 3, 0, 3, 6, 7, 0],
                    [0, 0, 0, 4, 0, 4, 0, 7, 9],
                    [0, 5, 0, 5, 5, 0, 6, 0, 9],                          
                    [0, 8, 8, 8, 0, 0, 8, 8, 9]])

incorrect=np.array([[9, 9, 9, 9, 9, 9, 9, 9, 9],
                    [8, 7, 6, 6, 6, 6, 6, 6, 9],
                    [8, 7, 5, 5, 5, 5, 5, 6, 9],
                    [8, 7, 4, 3, 2, 1, 5, 6, 9],
                    [8, 7, 4, 3, 2, 2, 5, 6, 9],
                    [8, 7, 4, 3, 3, 3, 5, 6, 9],
                    [8, 7, 4, 4, 4, 4, 5, 6, 9],
                    [8, 7, 7, 7, 7, 7, 7, 7, 9],                          
                    [8, 8, 8, 8, 8, 8, 8, 8, 9]])




[8 5 6 7 9]
